In [1]:
import os

NUM_THREADS = "1"

os.environ["OMP_NUM_THREADS"] = NUM_THREADS
os.environ["OPENBLAS_NUM_THREADS"] = NUM_THREADS
os.environ["MKL_NUM_THREADS"] = NUM_THREADS
os.environ["VECLIB_MAXIMUM_THREADS"] = NUM_THREADS
os.environ["NUMEXPR_NUM_THREADS"] = NUM_THREADS

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
# 
from TEST_OLD_UTILS.SHIPS_preprocess import SHIPS_train_test_split, calc_d24_VMAX, fore_hr_averaging, SHIPS_train_test_shuffle_CLASS
from TEST_OLD_UTILS.SHIPS_preprocess import load_processed_SHIPS, calculate_class_weights, get_RI_classes
from TEST_OLD_UTILS.SHIPS_ML_model_funcs import apply_class_label, calc_CM_stats, get_scores_class_rept, get_roc_auc, get_feature_importances_RF
from TEST_OLD_UTILS.SHIPS_ML_model_funcs import get_confusion_matrix_RF, get_scores_best_params_RF, create_gridsearch_RF, get_train_test_split
from TEST_OLD_UTILS.SHIPS_ML_model_funcs import calc_AUPD, calculate_PD_curves
from TEST_OLD_UTILS.SHIPS_plotting import plot_roc_curve, plot_precision_recall_vs_threshold,add_model_results,make_performance_diagram_background
from TEST_OLD_UTILS.SHIPS_plotting import plot_CSI_vs_bias, plot_basic_score_basin, plot_PD_curves
#
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve,roc_auc_score,confusion_matrix,accuracy_score,precision_score,recall_score,classification_report
from sklearn.metrics import precision_recall_curve, auc, f1_score, fbeta_score
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline
import matplotlib.colors


##### Experiment info for both models

In [3]:
fpath_load = 'DATA/ML_model_results/TRAINING/'
train_yr_start = 2005
train_yr_end = 2018
mask_TYPE = 'SIMPLE'
is_stand = True
if is_stand:
    stand_str = 'STANDARDIZED'
else:
    stand_str = 'no_STANDARD'
RI_thresh = 30
wts_str = 'default'
score = 'f1_weighted'
N_samples = 15

##### RF experiment info

In [4]:
RF_crit = 'gini'
RF_path_suff = 'OLD_FUNCS_TRAIN_{RF_crit}_SHIPS_SIMPLE_RI_vs_no_RI_{train_yr_start}-{train_yr_end}_{mask_TYPE}_MASK_'\
'{stand_str}_RI_thresh_{RI_thresh}weights_{wts_str}_wts_{N_samples}_samples_{score}.csv'.format(RF_crit=RF_crit,
                           train_yr_start=train_yr_start,train_yr_end=train_yr_end,mask_TYPE=mask_TYPE,
                           stand_str=stand_str,RI_thresh=RI_thresh,wts_str=wts_str,N_samples=N_samples,score=score)

##### RF Oversampling info


In [5]:
RF_path_suff_ov = 'OLD_FUNCS_TRAIN_OVERSAMPLE_{RF_crit}_SHIPS_SIMPLE_RI_vs_no_RI_{train_yr_start}-{train_yr_end}_{mask_TYPE}_MASK_'\
'{stand_str}_RI_thresh_{RI_thresh}weights_{wts_str}_{N_samples}_samples_{score}.csv'.format(RF_crit=RF_crit,
                           train_yr_start=train_yr_start,train_yr_end=train_yr_end,mask_TYPE=mask_TYPE,
                           stand_str=stand_str,RI_thresh=RI_thresh,wts_str='None',N_samples=N_samples,score=score)

##### LR experiment info
We'll just use `lbfgs` since it's so similar to `sag` \ `saga`

In [6]:
LR_model = 'lbfgs'
LR_path_suff = 'TRAIN_{LR_model}_SHIPS_SIMPLE_RI_vs_no_RI_{train_yr_start}-{train_yr_end}_{mask_TYPE}_MASK_'\
'{stand_str}_RI_thresh_{RI_thresh}weights_{wts_str}_wts_{N_samples}_samples_{score}.csv'.format(LR_model=LR_model,
                           train_yr_start=train_yr_start,train_yr_end=train_yr_end,mask_TYPE=mask_TYPE,
                           stand_str=stand_str,RI_thresh=RI_thresh,wts_str=wts_str,N_samples=N_samples,score=score)

##### Load confusion matrices

In [7]:
cm_RF = pd.read_csv(fpath_load+'RF/Conf_Matrix'+RF_path_suff)
cm_RF['Model'] = 'Random Forest (class wt)'
#
cm_RF_ov = pd.read_csv(fpath_load+'RF/Conf_Matrix'+RF_path_suff_ov)
cm_RF_ov['Model'] = 'Random Forest (oversampling)'

#
cm_LR = pd.read_csv(fpath_load+'LOGISTIC/Conf_Matrix'+LR_path_suff)
cm_LR['Model'] = 'Logistic Reg.'
cm_res = pd.concat([cm_RF,cm_RF_ov,cm_LR])


#### How are we selecting our best model hyperparameters? 
* Option 1: Hyperparameters corresponding to best model, determined by highest Threat Score / CSI
* Option 2: We pick the MODE (most commonly occuring) hyperparameter

For 3/4 hyperparameters, Option 1 and Option 2 produce the same answer (yay).  The difference is <code>n_estimators</code>.

In [14]:
choose_max = 'best'
max_threat = cm_res.set_index(['Category','BASIN','Model'])
threat_ind = max_threat.xs((1)).reset_index().groupby(['BASIN','Model']).max()
#if choose_max == 'best':
 #   isel = max_threat.xs((1,'ALL','Random Forest')).reset_index()['Threat'].idxmax()
  #  max_threat_sel = max_threat.xs((1,'ALL','Random Forest')).reset_index().iloc[isel]
#fig_format = 'pdf'